In [5]:
import numpy as np 
import pandas as pd

#csv 불러오기
data = pd.read_csv('dataSets/mediapipe.csv')
column = data.columns
data= data.iloc[:,1:]

data

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20
0,0.186389,0.251209,0.360036,0.473493,0.558989,0.477335,0.584770,0.546844,0.489406,0.478788,...,-0.131906,-0.135522,0.066758,-0.025118,-0.077841,-0.073980,0.103358,0.062316,0.044060,0.053988
1,0.278550,0.379559,0.503200,0.614705,0.689562,0.593821,0.788051,0.738457,0.665284,0.606631,...,0.003020,0.037178,-0.068465,0.027310,0.080409,0.080983,0.052543,0.116205,0.131019,0.124803
2,0.308312,0.411668,0.563913,0.646851,0.641663,0.658026,0.693098,0.607341,0.560114,0.634870,...,-0.287576,-0.243311,-0.157315,-0.286334,-0.237843,-0.168681,-0.145796,-0.226071,-0.218763,-0.189064
3,0.390868,0.442492,0.553352,0.640000,0.681346,0.600816,0.750522,0.711627,0.661650,0.607507,...,-0.038061,-0.030927,0.080941,0.048887,0.010194,0.010052,0.167412,0.143311,0.118499,0.114024
4,0.353267,0.430703,0.536424,0.644733,0.749795,0.665721,0.771581,0.729622,0.672792,0.661523,...,-0.171879,-0.182803,0.065070,-0.055632,-0.121745,-0.120124,0.110215,0.004009,-0.051153,-0.055663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.859817,0.795775,0.678057,0.571716,0.470835,0.504596,0.339868,0.238010,0.161133,0.516410,...,-0.239632,-0.226297,0.046558,-0.023056,-0.079792,-0.085790,0.091502,0.110888,0.107783,0.123058
995,0.823255,0.783856,0.699378,0.634949,0.609117,0.511456,0.365304,0.284603,0.203986,0.513518,...,-0.312959,-0.263820,-0.103970,-0.271309,-0.309975,-0.271905,-0.119773,-0.162357,-0.161356,-0.129625
996,0.230985,0.297674,0.381753,0.430198,0.422114,0.521491,0.651861,0.714603,0.772838,0.507179,...,-0.198416,-0.147612,-0.068035,-0.222042,-0.226498,-0.178866,-0.114816,-0.168460,-0.186807,-0.190323
997,0.298062,0.293335,0.336547,0.398895,0.448928,0.504882,0.655563,0.747907,0.839593,0.518714,...,-0.250979,-0.176464,-0.082016,-0.231829,-0.248048,-0.207366,-0.062716,-0.127224,-0.129845,-0.108340


In [ ]:
from typing import Dict

class Gesture_Predict:

    def __init__(self, limit=1):
        
        self.angle_data = pd.DataFrame(columns=['angle'+str(x) for x in range(0,18,1)])
        self.limit = limit
        self.predict = []

    def refresh_data(self):
        self.angle_data = pd.DataFrame(columns=['angle'+str(x) for x in range(0,18,1)])
        self.predict = []

    # =======================================================================================
    # Compute Angles
    # =======================================================================================
    def compute_angles(self, img_list, user_ids):

        for idx, file in enumerate(img_list):
        
            joint = file

            # landmark를 통해 벡터 생성
            v1 = joint[[0,1,2,3,0,5,6,7,0, 9,10,11, 0,13,14,15, 0,17,18,19],:] 
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]

            # v1->v2 벡터
            v = v2 - v1 # [20,3]

            # 벡터정규화
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # 벡터간의 cos유사도 계산
            sim1 = np.einsum('nt,nt->n',
                v[[0,5, 9,13],:], 
                v[[3,9,13,17],:])
            sim2 = np.einsum('nt,nt->n',
                -v[[1,2,4,5,6,8, 9,10,12,13,14,16,17,18],:], 
                v[[2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])

            sim = np.concatenate((sim1, sim2),axis=0)

            # 
            self.angle_data.loc[str(user_ids[idx])] = sim


    # =======================================================================================
    # Custom Unsupervised Learning Algorithm
    # =======================================================================================
    def fit_predict(self, df):
        distance = pd.DataFrame(columns=['x','y', 'value'])
        count=1
        for i in range(0,len(df.index),1):
            for j in range(i+1,len(df.index),1):
                row1=np.array(df.iloc[i,:])
                row2=np.array(df.iloc[j,:])
                distance=distance.append({'x': df.index[i], 'y': df.index[j],'value':((row1-row2)**2).sum()}, ignore_index=True)
    
        distance=distance[distance.value<self.limit] # 거리 제한

        for k in distance.itertuples():
            try:
                x = distance.loc[distance.value.idxmin()]
                self.predict.append([x.x, x.y])
                count += 1
                distance = distance[distance['x'] != x.x]
                distance = distance[distance['x'] != x.y]
                distance = distance[distance['y'] != x.x]
                distance = distance[distance['y'] != x.y]
            except:
                break
        return self.predict


    def do_predict(self, gesture_dict:Dict):
        self.refresh_data()

        img_list=list(gesture_dict.values())
        err = self.compute_angles(img_list, list(gesture_dict.keys()))

        if err is not None:
            return (True, err)

        df = self.angle_data

        self.fit_predict(df)

        return self.predict